### Cluster generate

In [1]:
# ==========================================
# Cell 1: 聚类分析 (Clustering)
# ==========================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import linkage, fcluster
from pathlib import Path
from typing import Tuple, List, Dict
import warnings

# 忽略警告
warnings.filterwarnings('ignore')

# 1. 核心聚类算法
def auto_select_k_and_cluster(X_data: np.ndarray, max_k: int = 10) -> np.ndarray:
    """自动选择最佳K值并返回聚类标签 (基于肘部法则-最大降幅)"""
    n_samples = len(X_data)
    if n_samples < 3:
        return np.zeros(n_samples, dtype=int)

    Z = linkage(X_data, method='ward')
    
    wcss_list = []
    valid_ks = list(range(2, min(max_k, n_samples) + 1))
    
    for k in valid_ks:
        labels = fcluster(Z, k, criterion='maxclust')
        wcss = 0
        for i in range(1, k + 1):
            cluster_points = X_data[labels == i]
            if len(cluster_points) > 0:
                center = cluster_points.mean(axis=0)
                wcss += np.sum((cluster_points - center) ** 2)
        wcss_list.append(wcss)
    
    if len(wcss_list) < 2:
        best_k = valid_ks[0]
    else:
        deltas = [wcss_list[i] - wcss_list[i+1] for i in range(len(wcss_list)-1)]
        best_idx = np.argmax(deltas)
        best_k = valid_ks[best_idx + 1]
    
    return fcluster(Z, best_k, criterion='maxclust') - 1

def run_clustering_single_metric(df_source: pd.DataFrame, l2_name: str, need_scale: bool = False) -> pd.Series:
    """处理单条L2政策"""
    # === 修改点1：严格限制年份 2005-2023 ===
    df = df_source[(df_source['TIME_PERIOD'] >= 2005) & (df_source['TIME_PERIOD'] <= 2023)]
    
    X = df.set_index(['REF_AREA', 'TIME_PERIOD'])[l2_name].unstack().fillna(0)
    
    if X.empty:
        return pd.Series(dtype=int)

    if need_scale:
        scaler = MinMaxScaler()
        if X.values.max() == X.values.min():
            X_vals = X.values
        else:
            X_vals = scaler.fit_transform(X)
    else:
        X_vals = X.values
        
    labels = auto_select_k_and_cluster(X_vals)
    return pd.Series(labels, index=X.index, name='ClusterID')

# 2. 数据 IO
def load_data(data_dir: Path):
    df_b = pd.read_parquet(data_dir / "2-1-country_breadth.parquet")
    df_i = pd.read_parquet(data_dir / "2-1-country_intensity.parquet")
    l2_list = [c for c in df_b.columns if c not in {'REF_AREA', 'TIME_PERIOD'}]
    return df_b, df_i, l2_list

def save_clustered_data(cluster_results: Dict[str, pd.Series], df_source: pd.DataFrame, output_path: Path):
    records = []
    # 同样确保源数据也是 2005-2023，以便保存时一致
    df_source_filtered = df_source[(df_source['TIME_PERIOD'] >= 2005) & (df_source['TIME_PERIOD'] <= 2023)]
    
    for l2, clusters in cluster_results.items():
        if clusters.empty: continue
        
        sub = df_source_filtered[['REF_AREA', 'TIME_PERIOD', l2]].copy()
        sub.columns = ['国家', '年份', '占比'] # 统一列名
        sub['聚类ID'] = sub['国家'].map(clusters)
        sub['L2政策'] = l2
        sub['L2政策中文名'] = l2 # 占位，确保列存在
        
        sub = sub.dropna(subset=['聚类ID'])
        sub['聚类ID'] = sub['聚类ID'].astype(int)
        records.append(sub)
        
    if records:
        final_df = pd.concat(records)
        final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"✅ 已保存: {output_path.name}")

# 3. 执行
base_dir = Path.cwd().parent
data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
print(f"数据目录: {data_dir}")

df_b, df_i, l2_list = load_data(data_dir)

print("正在执行 Breadth 聚类 (2005-2023)...")
b_results = {l2: run_clustering_single_metric(df_b, l2, need_scale=False) for l2 in l2_list}
save_clustered_data(b_results, df_b, data_dir / "3-1-L2_Policy_Clustering_Breadth.csv")

print("正在执行 Intensity 聚类 (2005-2023)...")
i_results = {l2: run_clustering_single_metric(df_i, l2, need_scale=True) for l2 in l2_list}
save_clustered_data(i_results, df_i, data_dir / "3-1-L2_Policy_Clustering_Intensity.csv")

数据目录: f:\Desktop\科研项目\1.负责科研项目\Climate Policy\CAMPF_Supplementary\data
正在执行 Breadth 聚类 (2005-2023)...
✅ 已保存: 3-1-L2_Policy_Clustering_Breadth.csv
正在执行 Intensity 聚类 (2005-2023)...
✅ 已保存: 3-1-L2_Policy_Clustering_Intensity.csv


### Table generate（生成可编辑版的PPT，和老师讨论修改）

In [2]:
# ==========================================
# Cell 1: 生成全量数据 -> 模拟 Cell 3 排序 -> 锁定顺序
# ==========================================
import pandas as pd
import numpy as np
import json
from pathlib import Path
from typing import Dict, List, Any

# --- 路径配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
OUTPUT_CSV = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
OUTPUT_JSON = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"

# --- 1. 特征计算 (来自 Cell 12) ---
def calculate_cluster_metrics(df_sub: pd.DataFrame, is_intensity: bool) -> Dict[int, Dict[str, Any]]:
    th_high = 6.0 if is_intensity else 0.6
    th_med = 3.0 if is_intensity else 0.3
    th_slope = 1.5 if is_intensity else 0.15

    df_calc = df_sub[(df_sub['年份'] >= 2005) & (df_sub['年份'] <= 2023)]
    # 注意：groupby 默认会按 key 排序，这里保证了 trends 的 index 是有序的
    trends = df_calc.groupby(['聚类ID', '年份'])['占比'].mean().unstack()
    
    features = {}
    for cid, row in trends.iterrows():
        ts = row.dropna()
        if len(ts) < 2:
            features[cid] = {'Starting': 'Low', 'Trend': 'Stable', 'Ending': 'Low', 'MeanStart': 0}
            continue
            
        start_val = ts.iloc[:3].mean()
        end_val = ts.iloc[-3].mean()
        slope = end_val - start_val
        std = ts.std()
        
        if start_val > th_high: s_lbl = 'High'
        elif start_val > th_med: s_lbl = 'Medium'
        else: s_lbl = 'Low'
        
        if end_val > th_high: e_lbl = 'High'
        elif end_val > th_med: e_lbl = 'Medium'
        else: e_lbl = 'Low'
        
        if slope > th_slope: t_lbl = 'Rise'
        elif slope < -0.05: t_lbl = 'Decline'
        elif std > (1.0 if is_intensity else 0.1): t_lbl = 'Fluctuate'
        else: t_lbl = 'Stable'
        
        features[cid] = {
            'Starting': s_lbl, 'Trend': t_lbl, 'Ending': e_lbl, 'MeanStart': start_val
        }
    return features

# --- 2. 排序逻辑 (100% 复刻 Cell 3) ---
def sort_clusters_original_logic(sort_df: pd.DataFrame) -> pd.DataFrame:
    """完全使用 Cell 3 的排序 Key 逻辑"""
    orders = {
        'Starting': {'Low': 0, 'Medium': 1, 'High': 2},
        'Ending': {'Low': 0, 'Medium': 1, 'High': 2},
        'Trend': {'Rise': 0, 'Stable': 1, 'Fluctuate': 2, 'Decline': 3}
    }
    
    df = sort_df.copy()
    df['sort_key'] = df.apply(lambda r: (
        orders['Starting'].get(r['Starting'], 99),
        orders['Ending'].get(r['Ending'], 99),
        orders['Trend'].get(r['Trend'], 99),
        r.get('MeanStart', 0)
    ), axis=1)
    
    return df.sort_values('sort_key')

# --- 3. 主执行流程 ---
def process_and_lock_strict():
    print("🚀 开始执行：生成数据 -> 全量排序 -> 锁定 ID")
    
    all_records = []
    
    # === 步骤 A: 生成所有数据 (模拟 Cell 12 的写入过程) ===
    tasks = [
        ("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth", False),
        ("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity", True)
    ]

    for fname, m_type, is_intensity in tasks:
        fpath = DATA_DIR / fname
        if not fpath.exists():
            print(f"⚠️ 跳过文件: {fname}")
            continue

        df_raw = pd.read_csv(fpath, encoding='utf-8-sig')
        
        # 严格按照 Cell 12 的循环顺序生成记录
        for l2 in df_raw['L2政策中文名'].dropna().unique():
            l2_data = df_raw[df_raw['L2政策中文名'] == l2]
            feats_map = calculate_cluster_metrics(l2_data, is_intensity)
            
            # 模拟 Cell 12 的 iterrows 展开
            for _, row in l2_data.iterrows():
                cid = row['聚类ID']
                if cid in feats_map:
                    f = feats_map[cid]
                    all_records.append({
                        'L2政策中文名': l2,
                        '国家': row['国家'],
                        '聚类ID': cid,
                        'Starting': f['Starting'],
                        'Trend': f['Trend'],
                        'Ending': f['Ending'],
                        'MeanStart': f['MeanStart'],
                        'Type': m_type
                    })

    if not all_records:
        print("❌ 没有生成任何数据")
        return

    # === 步骤 B: 构建 DataFrame 并去重 (模拟 Cell 12 的输出) ===
    df_full = pd.DataFrame(all_records).drop_duplicates(subset=['L2政策中文名', '国家', 'Type'])
    
    # 保存 CSV (这是 Cell 3 读取的对象)
    df_full.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"✅ CSV 已保存: {OUTPUT_CSV.name}")

    # === 步骤 C: 模拟 Cell 3 的读取和排序过程 ===
    # 为了绝对严谨，我们直接对 df_full 进行操作，就像 Cell 3 读进来一样
    order_map = {}
    
    # 1. 筛选 metric (Cell 3 是分开跑的，这里我们循环跑)
    for m_type in ['Breadth', 'Intensity']:
        df_metric = df_full[df_full['Type'] == m_type].copy()
        if df_metric.empty: continue
        
        # 2. 全量排序 (Cell 3 的关键步骤: sort_clusters(df_meta))
        # 这会根据 Starting/Ending 等对整个表排序
        df_metric_sorted = sort_clusters_original_logic(df_metric)
        
        # 3. 循环 L2 提取 ID (Cell 3 的绘图循环)
        for l2 in df_metric_sorted['L2政策中文名'].unique():
            # 提取该 L2 下的所有行
            l2_sort_info = df_metric_sorted[df_metric_sorted['L2政策中文名'] == l2]
            
            # Cell 3 里的逻辑: l2_sort_info.sort_values('sort_key')['聚类ID'].unique()
            # 虽然 df_metric_sorted 已经是排好序的，但 Cell 3 里为了保险又 sort 了一次
            # 我们这里也照做，确保行为一致
            final_sorted_df = sort_clusters_original_logic(l2_sort_info)
            sorted_ids = final_sorted_df['聚类ID'].unique().tolist()
            
            order_map[f"{m_type}_{l2}"] = sorted_ids

    # === 步骤 D: 锁定 JSON ===
    with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
        json.dump(order_map, f, ensure_ascii=False, indent=2)
    print(f"✅ 顺序已严格锁定至 JSON: {OUTPUT_JSON.name}")

# 运行
process_and_lock_strict()

🚀 开始执行：生成数据 -> 全量排序 -> 锁定 ID
✅ CSV 已保存: 3-2-Automated_Recognition_Mode.csv
✅ 顺序已严格锁定至 JSON: 3-2-fixed_ppt_order.json


In [3]:
# ==========================================
# Cell 2: 读取 JSON 顺序绘图
# ==========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator, MultipleLocator
from pathlib import Path
import json
import matplotlib as mpl

# --- 基础配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
ORDER_FILE = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
OUTPUT_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic"

# --- 样式设置 ---
def setup_mpl_single2():
    mpl.rc('font', size=25)
    mpl.rcParams.update({
        'legend.fontsize': 'small', 'xtick.labelsize': 'small', 'ytick.labelsize': 'small',
        'lines.linewidth': 2, 'axes.linewidth': 2, 'xtick.major.pad': '12', 'ytick.major.pad': '12',
        'xtick.direction': 'in', 'ytick.direction': 'in', 'xtick.top': False, 'ytick.right': False,
        'mathtext.default': 'regular', 'axes.titlesize': 'small'
    })
setup_mpl_single2()

NATURE_COLORS = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488', '#F39B7F', 
    '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85', 
    '#E18727', '#20854E', '#0072B5', '#BC3C29', '#6F99AD'
]

def lighten_color(color, amount=0.7):
    try:
        c = mcolors.to_rgb(color)
        return tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
    except: return (0.5, 0.5, 0.5)

# --- 绘图核心 ---
def plot_fixed_order(l2_name, df_data, df_feats, sorted_ids, metric_label):
    # 数据准备
    df_plot = df_data[(df_data['年份'] >= 2005) & (df_data['年份'] <= 2023)]
    years = sorted(df_plot['年份'].unique())
    matrix = df_plot.pivot(index='年份', columns='国家', values='占比').reindex(years)
    overall_mean = matrix.mean(axis=1)
    y_max = matrix.max().max() * 1.15

    # 画布设置
    n_clusters = len(sorted_ids)
    n_cols, n_rows = 3, (n_clusters + 2) // 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 3.5 * n_rows), squeeze=False)
    
    # --- 严格按照 JSON 给定的 ID 顺序循环 ---
    for idx, cid in enumerate(sorted_ids):
        ax = axes[idx // n_cols, idx % n_cols]
        
        # 找对应的国家
        countries = [c for c in df_plot[df_plot['聚类ID'] == cid]['国家'].unique() if c in matrix.columns]
        if not countries: continue

        # 找标题 (从特征文件里读)
        feat_row = df_feats[(df_feats['L2政策中文名'] == l2_name) & (df_feats['聚类ID'] == cid)]
        if not feat_row.empty:
            r = feat_row.iloc[0]
            title_str = f"{r['Starting']}+{r['Trend']}+{str(r['Ending']).replace(' Share', '')}"
        else:
            title_str = f"Cluster {cid}"

        # 绘图操作
        color = NATURE_COLORS[idx % len(NATURE_COLORS)] # 颜色只跟顺序有关
        fill_color = lighten_color(color, 0.7)
        sub_matrix = matrix[countries]
        c_mean = sub_matrix.mean(axis=1)

        ax.plot(sub_matrix.index, sub_matrix.values, color=color, alpha=0.25, lw=1.2, zorder=1)
        ax.plot(c_mean.index, c_mean, marker='o', color=color, lw=2.5, ms=7,
                mfc=fill_color, mec=color, mew=1.8, label='Cluster Average',
                markevery=max(1, len(years)//10), zorder=10)
        ax.plot(overall_mean.index, overall_mean, color='#000000', ls='--', lw=2.5,
                label='Overall Average', alpha=0.85, zorder=9, dashes=(3, 2))

        ax.set_title(f"{title_str}\n({len(countries)} countries)", pad=15, ha='center')
        ax.set_ylim(bottom=-(y_max * 0.04), top=y_max)
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
        ax.tick_params(axis='x', rotation=90, labelsize=15, pad=5.5)
        
        if idx % n_cols == 0: ax.set_ylabel(metric_label)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend(loc='best', frameon=False, fontsize=12)

    # 清理多余子图
    for j in range(n_clusters, n_rows * n_cols): axes[j // n_cols, j % n_cols].axis('off')
    plt.subplots_adjust(wspace=0.25, top=0.85, bottom=0.15, hspace=0.45)
    
    # 保存
    safe_name = l2_name.replace("/", "_").replace(" ", "_").replace(":", "")
    out_dir_m = OUTPUT_DIR / f"Plots_{metric_label}"
    out_dir_m.mkdir(parents=True, exist_ok=True)
    out_path = out_dir_m / f"{safe_name}_{metric_label}_Sorted.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    print(f"  -> Saved: {out_path.name}")

# --- 主执行 ---
def main_plot(fname, m_label):
    print(f"\n>>> 开始绘图: {m_label}")
    if not ORDER_FILE.exists() or not FEATURE_FILE.exists():
        print("❌ 请先运行 Cell 1")
        return
        
    with open(ORDER_FILE, 'r', encoding='utf-8') as f:
        order_map = json.load(f)
    df_feats = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    
    fpath = DATA_DIR / fname
    if not fpath.exists(): return
    df_data = pd.read_csv(fpath, encoding='utf-8-sig')
    
    for l2 in df_data['L2政策中文名'].dropna().unique():
        key = f"{m_label}_{l2}"
        if key in order_map:
            plot_fixed_order(l2, df_data[df_data['L2政策中文名']==l2], df_feats, order_map[key], m_label)

# 运行
main_plot("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth")
main_plot("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity")


>>> 开始绘图: Breadth
  -> Saved: Buildings_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Buildings_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: Climate_governance_Breadth_Sorted.png
  -> Saved: Electricity_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Electricity_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: Fossil_fuel_production_policies_Breadth_Sorted.png
  -> Saved: GHG_emissions_data_and_reporting_Breadth_Sorted.png
  -> Saved: GHG_emissions_targets_Breadth_Sorted.png
  -> Saved: Industry_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Industry_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: International_climate_co-operation_Breadth_Sorted.png
  -> Saved: International_public_finance_Breadth_Sorted.png
  -> Saved: Public_Research,_Development_and_Demonstration_Breadth_Sorted.png
  -> Saved: Transport_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Transport_–_non_market-based_instruments_Brea

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator, MultipleLocator
from pathlib import Path
import io
import json
import matplotlib as mpl
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

# ==========================================
# 1. 样式与基础配置 (保持不动)
# ==========================================
def setup_mpl_single2():
    # 针对小尺寸图片(4英寸)优化字号
    mpl.rc('font', size=12) 
    mpl.rcParams.update({
        'legend.fontsize': 9, 
        'xtick.labelsize': 10, 
        'ytick.labelsize': 10,
        'lines.linewidth': 2, 
        'axes.linewidth': 1.2,
        'xtick.major.pad': '3', 
        'ytick.major.pad': '3',
        'xtick.direction': 'in', 
        'ytick.direction': 'in',
        'xtick.top': False, 
        'ytick.right': False,
        'mathtext.default': 'regular', 
        'axes.titlesize': 11
    })
setup_mpl_single2()

NATURE_COLORS = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488', '#F39B7F', 
    '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85', 
    '#E18727', '#20854E', '#0072B5', '#BC3C29', '#6F99AD'
]

def lighten_color(color, amount=0.7):
    try:
        c = mcolors.to_rgb(color)
        c = tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
        return c
    except: return color

# ==========================================
# 2. 核心绘图函数 (保持不动)
# ==========================================
def create_single_cluster_img(l2_data, cid, years, matrix, overall_mean, y_max, metric_label, color_idx, show_ylabel):
    """
    生成单个聚类的小图。
    - show_ylabel: True 则显示 Y 轴标题，False 则隐藏（但保留空间以对齐）
    """
    countries = l2_data[l2_data['聚类ID'] == cid]['国家'].unique()
    countries = [c for c in countries if c in matrix.columns]
    
    if not countries: return None

    # 画布尺寸：设定为 PPT 中图片的目标尺寸 (宽高比约 1.5)
    # 稍微加大一点，利用 PPT 空间
    fig, ax = plt.subplots(figsize=(4.2, 2.8))
    
    color = NATURE_COLORS[color_idx % len(NATURE_COLORS)]
    fill_color = lighten_color(color, 0.7)
    
    sub_matrix = matrix[countries]
    c_mean = sub_matrix.mean(axis=1)
    
    # --- 绘图 ---
    ax.plot(sub_matrix.index, sub_matrix.values, color=color, alpha=0.25, lw=1.0, zorder=1)
    ax.plot(c_mean.index, c_mean, marker='o', color=color, lw=2.0, ms=5,
            mfc=fill_color, mec=color, mew=1.5, label='Cluster Avg',
            markevery=max(1, len(years)//10), zorder=10)
    ax.plot(overall_mean.index, overall_mean, color='#000000', ls='--', lw=2.0,
            label='Overall Avg', alpha=0.85, zorder=9, dashes=(3, 2))
    
    # --- 坐标轴设置 ---
    # Y轴向下延伸 4%
    y_neg_padding = y_max * 0.04
    ax.set_ylim(bottom=-y_neg_padding, top=y_max)
    
    ax.xaxis.set_major_locator(MultipleLocator(2))
    ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
    ax.tick_params(axis='x', rotation=90) # 年份竖排
    
    # === 关键：Y轴标签控制 ===
    if show_ylabel:
        ax.set_ylabel(metric_label)
    else:
        ax.set_ylabel("") # 不写文字，但下面固定边距会保留位置
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # 图例
    ax.legend(loc='best', frameon=False, fontsize=8, handlelength=1.5, markerscale=0.8)
    
    # === 关键：使用固定边距代替 tight_layout ===
    # 这样所有图的绘图区域（坐标轴框）大小完全一致
    plt.subplots_adjust(left=0.18, right=0.97, top=0.95, bottom=0.25)
    
    # 保存
    img_stream = io.BytesIO()
    plt.savefig(img_stream, format='png', dpi=200, transparent=True)
    plt.close()
    img_stream.seek(0)
    return img_stream

# ==========================================
# 3. 数据加载与排序 (修改：优先读取JSON)
# ==========================================
def load_features(feature_file: Path, metric_type: str):
    if not feature_file.exists(): return pd.DataFrame()
    df = pd.read_csv(feature_file, encoding='utf-8-sig')
    df = df[df['Type'] == metric_type].copy()
    return df

def load_json_order(json_file: Path):
    if not json_file.exists():
        print(f"⚠️ JSON 顺序文件未找到: {json_file}")
        return {}
    with open(json_file, 'r', encoding='utf-8') as f:
        return json.load(f)

# ==========================================
# 4. PPT 生成主程序 (集成 JSON 排序)
# ==========================================
def generate_ppt_grid(csv_file: Path, feature_file: Path, json_file: Path, output_ppt: Path, metric_label: str):
    if not csv_file.exists(): return

    print(f"\n>>> 正在生成 PPT (JSON排序版): {output_ppt.name} ...")
    
    df_data = pd.read_csv(csv_file, encoding='utf-8-sig')
    df_data = df_data[(df_data['年份'] >= 2005) & (df_data['年份'] <= 2023)]
    
    # 加载数据
    df_features = load_features(feature_file, metric_label)
    order_map = load_json_order(json_file) # 核心：加载 JSON
    
    if df_features.empty: return

    # PPT 初始化
    prs = Presentation()
    prs.slide_width = Inches(13.333)
    prs.slide_height = Inches(7.5)
    
    # === 布局参数 ===
    MARGIN_LEFT = 0.5
    MARGIN_TOP = 1.1   # 稍微往上提
    COL_GAP = 0.15     # 减小列间距
    ROW_GAP = 0.4      # 行间距
    
    PLOT_WIDTH = 4.0
    PLOT_HEIGHT = 2.6  # 高度增加
    LABEL_HEIGHT = 0.35
    
    l2_list = df_data['L2政策中文名'].dropna().unique()
    
    for i, l2 in enumerate(l2_list):
        print(f"\r  [{i+1}/{len(l2_list)}] 处理: {l2[:20]}...", end="")
        
        l2_feats = df_features[df_features['L2政策中文名'] == l2]
        if l2_feats.empty: continue
        
        # === 核心逻辑修改：获取排序 ID ===
        key = f"{metric_label}_{l2}"
        if key in order_map:
            sorted_cids = order_map[key] # 优先：使用 JSON 里的顺序
        else:
            # 降级：如果没有 JSON 记录，退回到默认排序 (按 Cluster ID)
            # 或者你可以选择在这里加上你的旧排序逻辑，但通常 JSON 应该覆盖所有
            print(f" (⚠️ 无JSON记录, 使用默认ID顺序) ", end="")
            sorted_cids = sorted(l2_feats['聚类ID'].unique())

        l2_sub = df_data[df_data['L2政策中文名'] == l2]
        years = sorted(l2_sub['年份'].unique())
        matrix = l2_sub.pivot(index='年份', columns='国家', values='占比').reindex(years)
        overall_mean = matrix.mean(axis=1)
        y_max = matrix.max().max() * 1.15
        
        # 新建幻灯片
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        
        # A. 主标题
        tb = slide.shapes.add_textbox(Inches(0.5), Inches(0.1), Inches(12), Inches(0.8))
        p = tb.text_frame.paragraphs[0]
        p.text = l2
        p.font.size = Pt(24); p.font.bold = True; p.font.color.rgb = RGBColor(44, 62, 80)
        
        # --- 循环生成网格 ---
        for idx, cid in enumerate(sorted_cids):
            # 如果 JSON 里的 ID 在特征表里找不到（比如数据过滤了），跳过
            if cid not in l2_feats['聚类ID'].values: continue

            row, col = idx // 3, idx % 3
            
            curr_x = Inches(MARGIN_LEFT + col * (PLOT_WIDTH + COL_GAP))
            curr_y = Inches(MARGIN_TOP + row * (LABEL_HEIGHT + PLOT_HEIGHT + ROW_GAP))
            
            if curr_y + Inches(3) > prs.slide_height: break 
            
            # 准备文字
            feat_row = l2_feats[l2_feats['聚类ID'] == cid].iloc[0]
            s_t = feat_row['Starting']
            t_t = feat_row['Trend']
            e_t = str(feat_row['Ending']).replace(' Share', '') 
            
            countries = l2_sub[l2_sub['聚类ID'] == cid]['国家'].unique()
            label_text = f"{s_t}+{t_t}+{e_t} ({len(countries)} countries)"
            
            # B. 插入文本框
            tb_sub = slide.shapes.add_textbox(curr_x, curr_y, Inches(PLOT_WIDTH), Inches(LABEL_HEIGHT))
            tp = tb_sub.text_frame.paragraphs[0]
            tp.text = label_text
            tp.font.size = Pt(14); tp.font.bold = True; tp.alignment = PP_ALIGN.CENTER
            
            # C. 插入图片
            # === 关键：传入 show_ylabel 参数 ===
            show_y = (col == 0)
            
            img_stream = create_single_cluster_img(
                l2_sub, cid, years, matrix, overall_mean, y_max, metric_label, idx, 
                show_ylabel=show_y
            )
            
            if img_stream:
                pic_y = curr_y + Inches(LABEL_HEIGHT)
                slide.shapes.add_picture(img_stream, curr_x, pic_y, width=Inches(PLOT_WIDTH))
                
    prs.save(output_ppt)
    print(f"\n✅ PPT 生成完毕: {output_ppt.name}")

# ==========================================
# 5. 执行
# ==========================================
def main():
    base_dir = Path.cwd().parent
    data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
    output_dir = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "Human_catorgy"
    output_dir.mkdir(parents=True, exist_ok=True)
    
    feature_csv = data_dir / "3-2-Automated_Recognition_Mode.csv"
    json_file = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
    
    # 1. Breadth PPT
    generate_ppt_grid(
        data_dir / "3-1-L2_Policy_Clustering_Breadth.csv",
        feature_csv,
        json_file,
        output_dir / "Analysis_Report_Breadth_Editable.pptx",
        "Breadth"
    )
    
    # 2. Intensity PPT
    generate_ppt_grid(
        data_dir / "3-1-L2_Policy_Clustering_Intensity.csv",
        feature_csv,
        json_file,
        output_dir / "Analysis_Report_Intensity_Editable.pptx",
        "Intensity"
    )

if __name__ == "__main__":
    main()


>>> 正在生成 PPT (JSON排序版): Analysis_Report_Breadth_Editable.pptx ...
  [15/15] 处理: Transport – non mark...
✅ PPT 生成完毕: Analysis_Report_Breadth_Editable.pptx

>>> 正在生成 PPT (JSON排序版): Analysis_Report_Intensity_Editable.pptx ...
  [15/15] 处理: Transport – non mark...
✅ PPT 生成完毕: Analysis_Report_Intensity_Editable.pptx


### Wait Human change（重复跑一遍，核对一下，这次PPT直接跑的不可编辑版的）

In [5]:
import pandas as pd
from pptx import Presentation
from pathlib import Path
from pptx.util import Inches
from typing import List, Optional, Dict
import re
import json

# ==========================================
# 1. 强力清洗函数 (保留原逻辑)
# ==========================================
def normalize_title_robust(text: str) -> str:
    """
    终极清洗：转小写、替换所有类型横杠、去多余空格
    """
    if not isinstance(text, str): return str(text) if text is not None else ""
    text = text.replace('\u2013', '-').replace('\u2014', '-').replace('–', '-')
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

def normalize_spaces(text: str) -> str:
    """清理文本框内容中的特殊空格"""
    return re.sub(r'\s+', ' ', text).strip()

def get_slide_title(slide) -> Optional[str]:
    """获取幻灯片标题"""
    if slide.shapes.title and slide.shapes.title.text.strip():
        return slide.shapes.title.text.strip()
    
    candidates = []
    for shape in slide.shapes:
        if shape.has_text_frame and shape.text_frame.text.strip():
            candidates.append((shape.top, shape.text_frame.text.strip()))
    
    if candidates:
        candidates.sort(key=lambda x: x[0])
        if candidates[0][0] < Inches(2.0):
            return candidates[0][1]
    return None

# ==========================================
# 2. 核心：加载 JSON 锁定顺序
# ==========================================
def load_fixed_order(json_path: Path) -> Dict[str, List[int]]:
    if not json_path.exists():
        print(f"❌ JSON 顺序文件未找到: {json_path}")
        return {}
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# ==========================================
# 3. 核心同步逻辑 (利用 JSON 回填)
# ==========================================
def sync_ppt_with_json(ppt_path: Path, csv_path: Path, json_path: Path, target_type: str) -> None:
    print(f"\n{'='*50}")
    print(f"📂 处理文件: {ppt_path.name}")
    print(f"🎯 目标类型: {target_type}")
    print(f"🔐 顺序参照: {json_path.name}")
    print(f"{'='*50}")
    
    if not ppt_path.exists():
        print(f"❌ PPT文件不存在: {ppt_path}")
        return

    prs = Presentation(ppt_path)
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    order_map = load_fixed_order(json_path)
    
    if not order_map:
        print("❌ 无法加载顺序映射，终止操作。")
        return
    
    # === 关键1：创建匹配列 ===
    df['Match_Title'] = df['L2政策中文名'].apply(normalize_title_robust)
    
    # 同时也把 JSON 的 key 处理一下，方便查找
    # JSON key 格式: "Breadth_PolicyName" -> 我们存一个映射: clean_policy_name -> [ids]
    json_lookup = {}
    prefix = f"{target_type}_"
    for key, ids in order_map.items():
        if key.startswith(prefix):
            raw_policy_name = key[len(prefix):] # 去掉 "Breadth_" 前缀
            clean_name = normalize_title_robust(raw_policy_name)
            json_lookup[clean_name] = ids

    update_count = 0
    
    for slide_idx, slide in enumerate(prs.slides):
        raw_ppt_title = get_slide_title(slide)
        if not raw_ppt_title: continue
        
        # 处理 PPT 标题（有些标题可能包含括号里的 metric type，去掉它）
        title_pure = raw_ppt_title.replace(f"({target_type})", "").strip()
        clean_ppt_title = normalize_title_robust(title_pure)
        
        # 1. 查找 JSON 里的顺序
        if clean_ppt_title not in json_lookup:
            # 尝试模糊匹配 (比如 PPT 标题不完整)
            found = False
            for k in json_lookup.keys():
                if k in clean_ppt_title or clean_ppt_title in k:
                    clean_ppt_title = k
                    found = True
                    break
            if not found:
                # print(f"⚠️ [无顺序记录] PPT标题: '{raw_ppt_title}' - 跳过")
                continue
        
        sorted_cids = json_lookup[clean_ppt_title]
        
        # 2. 获取 PPT 里的标签框 (从左到右排序)
        # 筛选逻辑：有文本，且文本里包含 '+' 号 (特征标识)
        label_shapes = [s for s in slide.shapes if s.has_text_frame and "+" in s.text_frame.text]
        # 排序：先按 Top (行)，再按 Left (列)。允许多行布局。
        # 容差：Top 差异在 0.5 英寸内视为同一行
        label_shapes.sort(key=lambda s: (int(s.top / Inches(0.5)), s.left))
        
        if len(label_shapes) != len(sorted_cids):
            # 这种情况比较少见，可能是 PPT 被人手动删改了
            print(f"⚠️ [数量不符] {title_pure}: PPT有{len(label_shapes)}个框, JSON记录有{len(sorted_cids)}个聚类")
            continue
            
        # 3. 逐一回填
        for shape, cid in zip(label_shapes, sorted_cids):
            raw_text = normalize_spaces(shape.text_frame.text)
            
            # 解析 PPT 文本内容: "High + Stable + Low (xx countries)"
            parts = raw_text.split('+')
            if len(parts) >= 3:
                p1 = parts[0].strip()
                p2 = parts[1].strip()
                p3_full = parts[2].strip()
                
                # 处理 Ending 部分，去掉括号里的国家数
                final_e = p3_full
                last_paren = p3_full.rfind('(')
                if last_paren != -1:
                    final_e = p3_full[:last_paren].strip()
                
                # 清洗 "Share" 后缀
                new_s = p1.replace(' Share', '')
                new_t = p2
                new_e = final_e.replace(' Share', '')
                
                # 4. 更新 CSV
                # 定位条件：标准化的政策名 + 类型 + 聚类ID (这个ID来自 JSON，绝对准确)
                target_idx = df[(df['Match_Title'] == clean_ppt_title) & 
                                (df['Type'] == target_type) & 
                                (df['聚类ID'] == cid)].index
                
                if not target_idx.empty:
                    # 检查是否需要更新
                    old_vals = df.loc[target_idx[0], ['Starting', 'Trend', 'Ending']].values
                    new_vals = [new_s, new_t, new_e]
                    
                    if list(old_vals) != new_vals:
                        print(f" 🔄 [更新] {title_pure[:15]}... (ID:{cid})")
                        print(f"    旧: {old_vals} -> 新: {new_vals}")
                        
                        df.loc[target_idx, 'Starting'] = new_s
                        df.loc[target_idx, 'Trend'] = new_t
                        df.loc[target_idx, 'Ending'] = new_e
                        update_count += len(target_idx) # 可能一次更新多行(多个国家)

    # 清理
    if 'Match_Title' in df.columns: del df['Match_Title']
    
    # 保存
    if update_count > 0:
        try:
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f"✅ 保存成功！共更新了 {update_count} 行数据。")
        except PermissionError:
            print("❌ 保存失败！请先关闭 CSV 文件！")
    else:
        print("✨ 文件已是最新，无需更新。")

# ==========================================
# 4. 主程序
# ==========================================
def main():
    base_dir = Path.cwd().parent
    data_dir = base_dir / "data" if (base_dir / "data").exists() else Path.cwd() / "data"
    
    # 路径配置
    # 注意：这里假设你的人工修改后的 PPT 放在这个 Human_catorgy 文件夹下
    ppt_dir = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "Human_catorgy" /"Final_Edit"
    csv_file = data_dir / "3-2-Automated_Recognition_Mode.csv"
    json_file = data_dir / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
    
    # 1. 更新 Breadth
    sync_ppt_with_json(ppt_dir / "Analysis_Report_Breadth_Editable.pptx", csv_file, json_file, "Breadth")
    
    # 2. 更新 Intensity
    sync_ppt_with_json(ppt_dir / "Analysis_Report_Intensity_Editable.pptx", csv_file, json_file, "Intensity")

if __name__ == "__main__":
    main()


📂 处理文件: Analysis_Report_Breadth_Editable.pptx
🎯 目标类型: Breadth
🔐 顺序参照: 3-2-fixed_ppt_order.json
 🔄 [更新] Buildings – mar... (ID:0)
    旧: ['Low' 'Rise' 'Low'] -> 新: ['Low', 'Stable', 'Low']
 🔄 [更新] Buildings – mar... (ID:1)
    旧: ['Low' 'Rise' 'High'] -> 新: ['High', 'Rise', 'High']
 🔄 [更新] Buildings – non... (ID:1)
    旧: ['Low' 'Rise' 'High'] -> 新: ['Low', 'Rise', 'Low']
 🔄 [更新] Buildings – non... (ID:2)
    旧: ['Medium' 'Rise' 'High'] -> 新: ['Medium', 'Rise', 'Medium']
 🔄 [更新] Climate governa... (ID:2)
    旧: ['Low' 'Rise' 'High'] -> 新: ['Low', 'Fluctuate', 'High']
 🔄 [更新] Electricity – m... (ID:2)
    旧: ['Low' 'Rise' 'Medium'] -> 新: ['Medium', 'Rise', 'Medium']
 🔄 [更新] Electricity – m... (ID:1)
    旧: ['Medium' 'Rise' 'Medium'] -> 新: ['High', 'Fluctuate', 'High']
 🔄 [更新] Electricity – n... (ID:0)
    旧: ['Low' 'Rise' 'Medium'] -> 新: ['Low', 'Rise', 'Low']
 🔄 [更新] Electricity – n... (ID:2)
    旧: ['Low' 'Rise' 'High'] -> 新: ['Medium', 'Rise', 'Medium']
 🔄 [更新] Electricity – n... (ID

In [6]:
# ==========================================
# Cell 2: 读取 JSON 顺序绘图
# ==========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator, MultipleLocator
from pathlib import Path
import json
import matplotlib as mpl

# --- 基础配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
ORDER_FILE = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
OUTPUT_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic"

# --- 样式设置 ---
def setup_mpl_single2():
    mpl.rc('font', size=25)
    mpl.rcParams.update({
        'legend.fontsize': 'small', 'xtick.labelsize': 'small', 'ytick.labelsize': 'small',
        'lines.linewidth': 2, 'axes.linewidth': 2, 'xtick.major.pad': '12', 'ytick.major.pad': '12',
        'xtick.direction': 'in', 'ytick.direction': 'in', 'xtick.top': False, 'ytick.right': False,
        'mathtext.default': 'regular', 'axes.titlesize': 'small'
    })
setup_mpl_single2()

NATURE_COLORS = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488', '#F39B7F', 
    '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85', 
    '#E18727', '#20854E', '#0072B5', '#BC3C29', '#6F99AD'
]

def lighten_color(color, amount=0.7):
    try:
        c = mcolors.to_rgb(color)
        return tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
    except: return (0.5, 0.5, 0.5)

# --- 绘图核心 ---
def plot_fixed_order(l2_name, df_data, df_feats, sorted_ids, metric_label):
    # 数据准备
    df_plot = df_data[(df_data['年份'] >= 2005) & (df_data['年份'] <= 2023)]
    years = sorted(df_plot['年份'].unique())
    matrix = df_plot.pivot(index='年份', columns='国家', values='占比').reindex(years)
    overall_mean = matrix.mean(axis=1)
    y_max = matrix.max().max() * 1.15

    # 画布设置
    n_clusters = len(sorted_ids)
    n_cols, n_rows = 3, (n_clusters + 2) // 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 3.5 * n_rows), squeeze=False)
    
    # --- 严格按照 JSON 给定的 ID 顺序循环 ---
    for idx, cid in enumerate(sorted_ids):
        ax = axes[idx // n_cols, idx % n_cols]
        
        # 找对应的国家
        countries = [c for c in df_plot[df_plot['聚类ID'] == cid]['国家'].unique() if c in matrix.columns]
        if not countries: continue

        # 找标题 (从特征文件里读)
        feat_row = df_feats[(df_feats['L2政策中文名'] == l2_name) & (df_feats['聚类ID'] == cid)]
        if not feat_row.empty:
            r = feat_row.iloc[0]
            title_str = f"{r['Starting']}+{r['Trend']}+{str(r['Ending']).replace(' Share', '')}"
        else:
            title_str = f"Cluster {cid}"

        # 绘图操作
        color = NATURE_COLORS[idx % len(NATURE_COLORS)] # 颜色只跟顺序有关
        fill_color = lighten_color(color, 0.7)
        sub_matrix = matrix[countries]
        c_mean = sub_matrix.mean(axis=1)

        ax.plot(sub_matrix.index, sub_matrix.values, color=color, alpha=0.25, lw=1.2, zorder=1)
        ax.plot(c_mean.index, c_mean, marker='o', color=color, lw=2.5, ms=7,
                mfc=fill_color, mec=color, mew=1.8, label='Cluster Average',
                markevery=max(1, len(years)//10), zorder=10)
        ax.plot(overall_mean.index, overall_mean, color='#000000', ls='--', lw=2.5,
                label='Overall Average', alpha=0.85, zorder=9, dashes=(3, 2))

        ax.set_title(f"{title_str}\n({len(countries)} countries)", pad=15, ha='center')
        ax.set_ylim(bottom=-(y_max * 0.04), top=y_max)
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
        ax.tick_params(axis='x', rotation=90, labelsize=15, pad=5.5)
        
        if idx % n_cols == 0: ax.set_ylabel(metric_label)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend(loc='best', frameon=False, fontsize=12)

    # 清理多余子图
    for j in range(n_clusters, n_rows * n_cols): axes[j // n_cols, j % n_cols].axis('off')
    plt.subplots_adjust(wspace=0.25, top=0.85, bottom=0.15, hspace=0.45)
    
    # 保存
    safe_name = l2_name.replace("/", "_").replace(" ", "_").replace(":", "")
    out_dir_m = OUTPUT_DIR / f"Plots_{metric_label}"
    out_dir_m.mkdir(parents=True, exist_ok=True)
    out_path = out_dir_m / f"{safe_name}_{metric_label}_Sorted.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    print(f"  -> Saved: {out_path.name}")

# --- 主执行 ---
def main_plot(fname, m_label):
    print(f"\n>>> 开始绘图: {m_label}")
    if not ORDER_FILE.exists() or not FEATURE_FILE.exists():
        print("❌ 请先运行 Cell 1")
        return
        
    with open(ORDER_FILE, 'r', encoding='utf-8') as f:
        order_map = json.load(f)
    df_feats = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    
    fpath = DATA_DIR / fname
    if not fpath.exists(): return
    df_data = pd.read_csv(fpath, encoding='utf-8-sig')
    
    for l2 in df_data['L2政策中文名'].dropna().unique():
        key = f"{m_label}_{l2}"
        if key in order_map:
            plot_fixed_order(l2, df_data[df_data['L2政策中文名']==l2], df_feats, order_map[key], m_label)

# 运行
main_plot("3-1-L2_Policy_Clustering_Breadth.csv", "Breadth")
main_plot("3-1-L2_Policy_Clustering_Intensity.csv", "Intensity")


>>> 开始绘图: Breadth
  -> Saved: Buildings_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Buildings_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: Climate_governance_Breadth_Sorted.png
  -> Saved: Electricity_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Electricity_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: Fossil_fuel_production_policies_Breadth_Sorted.png
  -> Saved: GHG_emissions_data_and_reporting_Breadth_Sorted.png
  -> Saved: GHG_emissions_targets_Breadth_Sorted.png
  -> Saved: Industry_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Industry_–_non_market-based_instruments_Breadth_Sorted.png
  -> Saved: International_climate_co-operation_Breadth_Sorted.png
  -> Saved: International_public_finance_Breadth_Sorted.png
  -> Saved: Public_Research,_Development_and_Demonstration_Breadth_Sorted.png
  -> Saved: Transport_–_market-based_instruments_Breadth_Sorted.png
  -> Saved: Transport_–_non_market-based_instruments_Brea

In [7]:
# ==========================================
# Cell 3: 生成 PPT (Strict Order Matching)
# ==========================================
import pandas as pd
import json
from pathlib import Path
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor

# --- 路径配置 ---
BASE_DIR = Path.cwd().parent
# 优先查找父级 data，若不存在则查找当前目录下 data
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
ORDER_FILE = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
# 修改为目标相对路径
IMG_BASE_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic"
IMG_BASE_DIR.mkdir(parents=True, exist_ok=True)

def create_table(slide, df_policy: pd.DataFrame, sorted_ids: list, l2_name: str) -> None:
    """
    在幻灯片底部创建特征表格，严格按照 sorted_ids 顺序排列。
    Args:
        slide: pptx 幻灯片对象
        df_policy: 当前政策的数据集 DataFrame
        sorted_ids: 锁定的聚类ID列表
        l2_name: 政策名称
    """
    rows = []
    for cid in sorted_ids:
        row_data = df_policy[df_policy['聚类ID'] == cid]
        if row_data.empty: continue
        
        r = row_data.iloc[0]
        countries = df_policy[df_policy['聚类ID'] == cid]['国家'].unique()
        country_str = ", ".join(countries)
        if len(country_str) > 50: country_str = country_str[:47] + "..."
        
        rows.append([
            f"Cluster {cid}",
            f"{r['Starting']} -> {r['Trend']} -> {r['Ending']}",
            f"{r['MeanStart']:.3f}",
            f"{len(countries)} ({country_str})"
        ])

    if not rows: return

    # 表格参数：行数, 列数, 左, 上, 宽, 高
    table = slide.shapes.add_table(len(rows) + 1, 4, Inches(0.5), Inches(4.5), Inches(9.0), Inches(0.8)).table

    # 设置列宽
    widths = [1.2, 2.5, 1.2, 4.1]
    for idx, w in enumerate(widths):
        table.columns[idx].width = Inches(w)

    # 表头设置
    headers = ["ID", "Trajectory Features", "Mean Start", "Countries"]
    for i, h in enumerate(headers):
        cell = table.cell(0, i)
        cell.text = h
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(220, 220, 220)
        cell.text_frame.paragraphs[0].font.bold = True
        cell.text_frame.paragraphs[0].font.size = Pt(10)

    # 内容填充
    for r_idx, row_content in enumerate(rows):
        for c_idx, text in enumerate(row_content):
            cell = table.cell(r_idx + 1, c_idx)
            cell.text = str(text)
            cell.text_frame.paragraphs[0].font.size = Pt(9)
            cell.vertical_anchor = 3 # Middle

def generate_ppt_for_metric(metric_label: str) -> None:
    """
    生成指定 Metric (Breadth/Intensity) 的 PPT 报告。
    Args:
        metric_label: 评估指标名称
    """
    if not ORDER_FILE.exists() or not FEATURE_FILE.exists(): return

    with open(ORDER_FILE, 'r', encoding='utf-8') as f:
        order_map = json.load(f)
    
    df_all = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    df_feats = df_all[df_all['Type'] == metric_label]

    prs = Presentation()
    prs.slide_width, prs.slide_height = Inches(13.333), Inches(7.5)

    for l2 in df_feats['L2政策中文名'].unique():
        key = f"{metric_label}_{l2}"
        if key not in order_map: continue
            
        sorted_ids = order_map[key]
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        
        # 标题
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(12), Inches(0.5))
        tf = title_box.text_frame
        tf.text = f"{l2} ({metric_label})"
        tf.paragraphs[0].font.bold = True
        tf.paragraphs[0].font.size = Pt(24)
        
        # 图片路径处理
        safe_name = l2.replace("/", "_").replace(" ", "_").replace(":", "")
        img_path = IMG_BASE_DIR / f"Plots_{metric_label}" / f"{safe_name}_{metric_label}_Sorted.png"
        
        if img_path.exists():
            slide.shapes.add_picture(str(img_path), Inches(0.5), Inches(0.8), width=Inches(12.3))
        else:
            # 仅在找不到图片时在PPT内提示，不中断
            err_box = slide.shapes.add_textbox(Inches(4), Inches(3), Inches(5), Inches(1))
            err_box.text_frame.text = "Image Not Found"

        # 生成表格
        create_table(slide, df_feats[df_feats['L2政策中文名'] == l2], sorted_ids, l2)

    output_path = IMG_BASE_DIR / f"Report_{metric_label}_Sorted.pptx"
    prs.save(output_path)

# --- 执行 ---
generate_ppt_for_metric("Breadth")
generate_ppt_for_metric("Intensity")

### Update the right change（确保正确后重新排序一下）

##### Picture

In [8]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
from pathlib import Path
from matplotlib.ticker import MaxNLocator, MultipleLocator

# ==========================================
# 配置与路径
# ==========================================
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
ORDER_FILE = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
OUTPUT_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic"

NATURE_COLORS = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488', '#F39B7F', 
    '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85', 
    '#E18727', '#20854E', '#0072B5', '#BC3C29', '#6F99AD'
]

# ==========================================
# 1. 样式设置 (使用您指定的高级格式)
# ==========================================
def setup_mpl_single2():
    mpl.rc('font', size=25)
    mpl.rcParams.update({
        'legend.fontsize': 'small', 'xtick.labelsize': 'small', 'ytick.labelsize': 'small',
        'lines.linewidth': 2, 'axes.linewidth': 2, 'xtick.major.pad': '12', 'ytick.major.pad': '12',
        'xtick.direction': 'in', 'ytick.direction': 'in', 'xtick.top': False, 'ytick.right': False,
        'mathtext.default': 'regular', 'axes.titlesize': 'small'
    })

def lighten_color(color, amount=0.7):
    try:
        c = mcolors.to_rgb(color)
        return tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
    except: return (0.5, 0.5, 0.5)

# ==========================================
# 2. 排序逻辑
# ==========================================
def get_sort_key(row):
    """排序优先级: Starting > Ending > Trend > MeanStart"""
    orders = {
        'Starting': {'Low': 0, 'Medium': 1, 'High': 2},
        'Ending': {'Low': 0, 'Medium': 1, 'High': 2},
        'Trend': {'Rise': 0, 'Stable': 1, 'Fluctuate': 2, 'Decline': 3}
    }
    return (
        orders['Starting'].get(row['Starting'], 99),
        orders['Ending'].get(row['Ending'], 99),
        orders['Trend'].get(row['Trend'], 99),
        row.get('MeanStart', 0)
    )

# ==========================================
# 3. 绘图核心 (应用特定格式)
# ==========================================
def plot_fixed_order(l2_name, df_data, df_feats, sorted_ids, metric_label):
    # 数据准备
    df_plot = df_data[(df_data['年份'] >= 2005) & (df_data['年份'] <= 2023)]
    years = sorted(df_plot['年份'].unique())
    matrix = df_plot.pivot(index='年份', columns='国家', values='占比').reindex(years)
    overall_mean = matrix.mean(axis=1)
    y_max = matrix.max().max() * 1.15

    # 画布设置
    n_clusters = len(sorted_ids)
    n_cols, n_rows = 3, (n_clusters + 2) // 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 3.5 * n_rows), squeeze=False)
    
    for idx, cid in enumerate(sorted_ids):
        ax = axes[idx // n_cols, idx % n_cols]
        
        # 找对应的国家
        countries = [c for c in df_plot[df_plot['聚类ID'] == cid]['国家'].unique() if c in matrix.columns]
        if not countries: continue

        # 找标题
        feat_row = df_feats[(df_feats['L2政策中文名'] == l2_name) & (df_feats['聚类ID'] == cid)]
        if not feat_row.empty:
            r = feat_row.iloc[0]
            title_str = f"{r['Starting']}+{r['Trend']}+{str(r['Ending']).replace(' Share', '')}"
        else:
            title_str = f"Cluster {cid}"

        # 绘图操作
        color = NATURE_COLORS[idx % len(NATURE_COLORS)]
        fill_color = lighten_color(color, 0.7)
        sub_matrix = matrix[countries]
        c_mean = sub_matrix.mean(axis=1)

        ax.plot(sub_matrix.index, sub_matrix.values, color=color, alpha=0.25, lw=1.2, zorder=1)
        ax.plot(c_mean.index, c_mean, marker='o', color=color, lw=2.5, ms=7,
                mfc=fill_color, mec=color, mew=1.8, label='Cluster Average',
                markevery=max(1, len(years)//10), zorder=10)
        ax.plot(overall_mean.index, overall_mean, color='#000000', ls='--', lw=2.5,
                label='Overall Average', alpha=0.85, zorder=9, dashes=(3, 2))

        ax.set_title(f"{title_str}\n({len(countries)} countries)", pad=15, ha='center')
        ax.set_ylim(bottom=-(y_max * 0.04), top=y_max)
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
        ax.tick_params(axis='x', rotation=90, labelsize=15, pad=5.5)
        
        if idx % n_cols == 0: ax.set_ylabel(metric_label)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend(loc='best', frameon=False, fontsize=12)

    # 清理多余子图
    for j in range(n_clusters, n_rows * n_cols): axes[j // n_cols, j % n_cols].axis('off')
    plt.subplots_adjust(wspace=0.25, top=0.85, bottom=0.15, hspace=0.45)
    
    # 保存
    safe_name = l2_name.replace("/", "_").replace(" ", "_").replace(":", "")
    out_dir_m = OUTPUT_DIR / f"Plots_{metric_label}"
    out_dir_m.mkdir(parents=True, exist_ok=True)
    out_path = out_dir_m / f"{safe_name}_{metric_label}_Sorted.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()

# ==========================================
# 4. 主执行流程 (排序 -> 绘图)
# ==========================================
def main_resort_and_plot():
    setup_mpl_single2() # 应用样式
    
    if not FEATURE_FILE.exists():
        print(f"❌ 未找到特征文件: {FEATURE_FILE}")
        return

    print("🚀 第一步: 重新排序并锁定 JSON...")
    df_feats = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    order_map = {}
    
    # 1. 生成排序
    for metric in ['Breadth', 'Intensity']:
        sub_df = df_feats[df_feats['Type'] == metric].copy()
        if sub_df.empty: continue
        sub_df['sort_key'] = sub_df.apply(get_sort_key, axis=1)
        sub_df = sub_df.sort_values('sort_key')
        
        for l2 in sub_df['L2政策中文名'].unique():
            ids = sub_df[sub_df['L2政策中文名'] == l2]['聚类ID'].unique().tolist()
            order_map[f"{metric}_{l2}"] = ids
            
    with open(ORDER_FILE, 'w', encoding='utf-8') as f:
        json.dump(order_map, f, ensure_ascii=False, indent=2)
    print(f"✅ 顺序已更新至: {ORDER_FILE.name}")

    # 2. 重新绘图
    print("\n🚀 第二步: 使用新顺序重新绘图...")
    tasks = [('Breadth', '3-1-L2_Policy_Clustering_Breadth.csv'), 
             ('Intensity', '3-1-L2_Policy_Clustering_Intensity.csv')]
             
    for metric, fname in tasks:
        fpath = DATA_DIR / fname
        if not fpath.exists(): continue
        df_raw = pd.read_csv(fpath, encoding='utf-8-sig')
        
        print(f">>> 正在处理 {metric} ...")
        for l2 in df_raw['L2政策中文名'].dropna().unique():
            key = f"{metric}_{l2}"
            if key in order_map:
                plot_fixed_order(l2, df_raw[df_raw['L2政策中文名'] == l2], df_feats, order_map[key], metric)
                
    print("\n✨ 全部完成！图片已按最新排序重新生成。")

if __name__ == "__main__":
    main_resort_and_plot()

🚀 第一步: 重新排序并锁定 JSON...
✅ 顺序已更新至: 3-2-fixed_ppt_order.json

🚀 第二步: 使用新顺序重新绘图...
>>> 正在处理 Breadth ...
>>> 正在处理 Intensity ...

✨ 全部完成！图片已按最新排序重新生成。


##### PPT

In [9]:
import pandas as pd
import json
import io
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator, MultipleLocator
from pathlib import Path
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor

# --- 配置 ---
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
FEATURE_FILE = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
ORDER_FILE = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "3-2-fixed_ppt_order.json"
OUTPUT_DIR = DATA_DIR / "3-1-(3-2)Sorted_L2_Policy_Clustering_pic" / "Human_catorgy"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
NATURE_COLORS = ['#E64B35', "#6917C2", '#00A087', '#3C5488', '#F39B7F', '#8491B4', '#91D1C2', '#DC0000', '#7E6148', '#B09C85', '#E18727', '#20854E', '#0072B5', '#BC3C29', '#6F99AD']

def create_img_stream(df_sub: pd.DataFrame, cid: int, color_idx: int, y_max: float, show_y: bool) -> io.BytesIO:
    """生成单个内存图片流"""
    years = sorted(df_sub['年份'].unique())
    matrix = df_sub.pivot(index='年份', columns='国家', values='占比').reindex(years)
    if matrix.empty: return None
    
    fig, ax = plt.subplots(figsize=(4.2, 2.8))
    col = NATURE_COLORS[color_idx % 15]
    fill = tuple([c + (1 - c) * 0.7 for c in mcolors.to_rgb(col)])
    
    sub = matrix[df_sub[df_sub['聚类ID'] == cid]['国家'].unique()]
    ax.plot(sub.index, sub.values, color=col, alpha=0.25, lw=1)
    ax.plot(sub.mean(axis=1), marker='o', color=col, lw=2, ms=5, mfc=fill, zorder=10)
    
    ax.set_ylim(bottom=-(y_max*0.04), top=y_max)
    ax.xaxis.set_major_locator(MultipleLocator(2))
    ax.tick_params(axis='x', rotation=90)
    ax.set_ylabel("Metric" if show_y else "")
    ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
    plt.subplots_adjust(left=0.18, right=0.97, top=0.95, bottom=0.25)
    
    stream = io.BytesIO()
    plt.savefig(stream, format='png', dpi=200, transparent=True)
    plt.close()
    stream.seek(0)
    return stream

def generate_ppt(metric: str, csv_name: str):
    """基于 JSON 顺序生成可编辑 PPT"""
    out_ppt = OUTPUT_DIR / f"Analysis_Report_{metric}_Editable.pptx"
    df_raw = pd.read_csv(DATA_DIR / csv_name, encoding='utf-8-sig')
    df_raw = df_raw[(df_raw['年份'] >= 2005) & (df_raw['年份'] <= 2023)]
    df_feats = pd.read_csv(FEATURE_FILE, encoding='utf-8-sig')
    with open(ORDER_FILE) as f: order_map = json.load(f)

    prs = Presentation()
    prs.slide_width, prs.slide_height = Inches(13.333), Inches(7.5)
    
    print(f"正在生成: {out_ppt.name} ...")
    for l2 in df_feats[df_feats['Type'] == metric]['L2政策中文名'].unique():
        ids = order_map.get(f"{metric}_{l2}", [])
        if not ids: continue
            
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        tb = slide.shapes.add_textbox(Inches(0.5), Inches(0.1), Inches(12), Inches(0.8))
        tb.text_frame.text = l2
        tb.text_frame.paragraphs[0].font.size = Pt(24)
        
        l2_data = df_raw[df_raw['L2政策中文名'] == l2]
        y_max = l2_data['占比'].max() * 1.15
        
        for idx, cid in enumerate(ids):
            row, col = idx // 3, idx % 3
            if row * 3 + col >= 6: break # 简单限制一页最多6张
            
            # 文本标签
            fr = df_feats[(df_feats['L2政策中文名'] == l2) & (df_feats['聚类ID'] == cid)].iloc[0]
            txt = f"{fr['Starting']}+{fr['Trend']}+{str(fr['Ending']).replace(' Share','')} ({len(l2_data[l2_data['聚类ID']==cid]['国家'].unique())} countries)"
            
            x, y = Inches(0.5 + col * 4.15), Inches(1.1 + row * 3.1)
            tf = slide.shapes.add_textbox(x, y, Inches(4), Inches(0.35)).text_frame
            tf.text = txt; tf.paragraphs[0].alignment = PP_ALIGN.CENTER; tf.paragraphs[0].font.bold = True
            
            # 图片
            stream = create_img_stream(l2_data, cid, idx, y_max, col == 0)
            if stream: slide.shapes.add_picture(stream, x, y + Inches(0.35), width=Inches(4))

    prs.save(out_ppt)
    print(f"✅ 完成: {out_ppt.name}")

if __name__ == "__main__":
    generate_ppt("Breadth", "3-1-L2_Policy_Clustering_Breadth.csv")
    generate_ppt("Intensity", "3-1-L2_Policy_Clustering_Intensity.csv")

正在生成: Analysis_Report_Breadth_Editable.pptx ...
✅ 完成: Analysis_Report_Breadth_Editable.pptx
正在生成: Analysis_Report_Intensity_Editable.pptx ...
✅ 完成: Analysis_Report_Intensity_Editable.pptx


### 更新人工编写文件

In [10]:
import pandas as pd
import json
from pathlib import Path
from typing import Dict, Tuple

# ==========================================
# 1. 路径配置
# ==========================================
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" if (BASE_DIR / "data").exists() else Path.cwd() / "data"
CONFIG_FILE = DATA_DIR / "config_mappings.json"

# 输入文件
FILE_3_2 = DATA_DIR / "3-2-Automated_Recognition_Mode.csv"
FILE_3_1_B = DATA_DIR / "3-1-L2_Policy_Clustering_Breadth.csv"
FILE_3_1_I = DATA_DIR / "3-1-L2_Policy_Clustering_Intensity.csv"

# 输出文件
OUTPUT_FILE = DATA_DIR / "3-3-Visualizing_Models_by_Country.csv"

# ==========================================
# 2. 辅助函数
# ==========================================
def load_mappings(json_path: Path) -> Tuple[Dict[str, str], Dict[str, str]]:
    """
    加载映射文件，构建 L2名称->代码 和 L2名称->L1名称 的映射字典。
    
    Args:
        json_path: config_mappings.json 的路径
    Returns:
        (l2_name_to_code, l2_name_to_l1)
    """
    if not json_path.exists():
        return {}, {}
    
    with open(json_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # 1. 构建 L2 Name -> L2 Code (反转 level2_names)
    # 原始: {"LEV2_XXX": "Name"} -> 目标: {"Name": "LEV2_XXX"}
    l2_name_to_code = {v: k for k, v in config.get('level2_names', {}).items()}
    
    # 2. 构建 L2 Name -> L1 Name
    # 遍历 level_mapping (结构为 LEV3 -> {L2:..., L1:...})
    l2_name_to_l1 = {}
    for item in config.get('level_mapping', {}).values():
        l2_name = item.get('L2')
        l1_name = item.get('L1')
        if l2_name and l1_name:
            l2_name_to_l1[l2_name] = l1_name
            
    return l2_name_to_code, l2_name_to_l1

def format_ending(val: str) -> str:
    """格式化 Ending 字段，确保以 ' Share' 结尾"""
    s_val = str(val)
    if s_val.lower() == 'nan': return ""
    return s_val if s_val.endswith(" Share") else f"{s_val} Share"

# ==========================================
# 3. 主逻辑
# ==========================================
def generate_final_table():
    """读取 3-1 和 3-2 数据，结合映射文件，生成包含 Type 的最终宽表"""
    
    # 1. 准备数据与映射
    l2_code_map, l2_l1_map = load_mappings(CONFIG_FILE)
    
    if not FILE_3_2.exists():
        return
    
    df_features = pd.read_csv(FILE_3_2, encoding='utf-8-sig')
    
    tasks = [
        (FILE_3_1_B, "Breadth"),
        (FILE_3_1_I, "Intensity")
    ]
    
    all_data = []

    for fpath, type_label in tasks:
        if not fpath.exists(): continue
            
        # 读取面板数据 (3-1)
        df_panel = pd.read_csv(fpath, encoding='utf-8-sig')
        
        # 筛选对应类型的特征数据 (3-2)
        df_feats_sub = df_features[df_features['Type'] == type_label].copy()
        
        # 合并: 面板数据 + 特征数据
        # 连接键: L2政策中文名, 国家, 聚类ID
        merged = pd.merge(
            df_panel, 
            df_feats_sub[['L2政策中文名', '国家', '聚类ID', 'Starting', 'Trend', 'Ending']], 
            on=['L2政策中文名', '国家', '聚类ID'],
            how='inner'
        )
        
        # 明确指定 Type
        merged['Type'] = type_label
        
        # 映射新字段
        merged['L2政策'] = merged['L2政策中文名'].map(l2_code_map)
        merged['L1分类'] = merged['L2政策中文名'].map(l2_l1_map)
        merged['L1分类中文名'] = merged['L1分类'] # 示例中 L1 中文名与英文名一致
        
        # 格式化 Ending
        merged['Ending'] = merged['Ending'].apply(format_ending)
        
        # 收集结果
        all_data.append(merged)

    if not all_data:
        return

    # 2. 合并与字段整理
    df_final = pd.concat(all_data, ignore_index=True)
    
    # 定义最终列顺序 (包含 Type)
    target_cols = [
        'L2政策', 'L2政策中文名', 'L1分类', 'L1分类中文名', 
        '聚类ID', '国家', '年份', '占比', 
        'Starting', 'Trend', 'Ending', 'Type'
    ]
    
    # 确保列存在，并重新排序
    final_cols = [c for c in target_cols if c in df_final.columns]
    df_final = df_final[final_cols]
    
    # 排序优化体验
    df_final = df_final.sort_values(by=['Type', 'L2政策', '国家', '年份'])
    
    # 3. 保存
    df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

if __name__ == "__main__":
    generate_final_table()